In [8]:
import numpy as np
from keras.preprocessing import image
import cv2
from models import sample

In [9]:
model = sample.emotion_recognition((48,48,1))
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 batch_normalization_10 (Bat  (None, 46, 46, 32)       128       
 chNormalization)                                                
                                                                 
 activation_10 (Activation)  (None, 46, 46, 32)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 batch_normalization_11 (Bat  (None, 46, 46, 64)       256       
 chNormalization)                                          

In [10]:
emotion_classifier = model.load_weights('../trained_models/emotion_weights_30.hdf5')
face_detection = cv2.CascadeClassifier('../trained_models/haarcascade_frontalface_default.xml')
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}

In [11]:
cap = cv2.VideoCapture(0)

while True:

 _,cap_image = cap.read()
 cap_img_gray = cv2.cvtColor(cap_image, cv2.COLOR_BGR2GRAY)
 faces = face_detection.detectMultiScale(cap_img_gray, 1.3, 5)

 for (x,y,w,h) in faces:

   cv2.rectangle(cap_image, (x,y), (x+w,y+h),(255,0,0),2)
   roi_gray = cap_img_gray[y:y+h, x:x+w]
   roi_gray = cv2.resize(roi_gray, (48,48))
   img_pixels = image.img_to_array(roi_gray)
   img_pixels = np.expand_dims(img_pixels, axis=0)

   predictions = model.predict(img_pixels)
   emotion_label = np.argmax(predictions)

   emotion_prediction = label_dict[emotion_label]

   cv2.putText(cap_image, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1 )

   resize_image = cv2.resize(cap_image, (1000,700))
   cv2.imshow('Emotion',resize_image)

   if cv2.waitKey(1) == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 